<a href="https://colab.research.google.com/github/kaluznys/uczenie_maszynowe_UW/blob/main/pd4_um_uw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework Assignment: Analyzing and Plotting Bias in Penalized Regression**

-------------------------------

In this assignment, you will explore how **Ridge** and **Lasso** regression introduce **bias** into a model to reduce **variance**, and how the choice of the regularization parameter $\lambda$ affects this trade-off. The goal is to visualize and analyze the **bias-variance trade-off** and understand the conditions under which penalization helps or hinders model performance.

## **The Question**

**How does varying the regularization parameter $\lambda$ in Ridge and Lasso regression impact the trade-off between bias and variance?**

- Generate a synthetic dataset based on a **known** linear relationship:
  
  $$
  y = \beta_0 + \beta_1 x + \ldots + \epsilon
  $$

  where $$\epsilon \sim \mathcal{N}(0, \sigma^2),$$

  use a **high-dimensional** setting (e.g., 50 predictors) with only a few non-zero true coefficients to emphasize the effects of regularization. I stress, the $\beta_i$ coefficients should be known for this experiment and they should be mostly 0, with only a few non-zero parameters.

- Investigate how increasing $\lambda$ influences the model’s **bias**, **variance**, and **Mean Squared Error (MSE)**.
- Plot **Bias²**, **Variance**, and **MSE** on a single graph for both Ridge and Lasso models.
- Explain MSE decomposition into bias and variance. Read more on the MSE decomposition if you need to.

**Does the regularization lead to an optimal trade-off point where MSE is minimized? Explain why this point exists.**




**Expected Outcome:**
 - As $\lambda$ increases:
   - **Bias** increases (the model becomes too simple).
   - **Variance** decreases (the model becomes more stable).
   - **MSE** forms a **U-shape**, revealing the optimal trade-off.

- Analyze how **Ridge** and **Lasso** differ in terms of their bias-variance trade-offs.
- Discuss situations where one method may outperform the other, considering factors like **feature sparsity** and **multicollinearity**.


## **Colab Notebook Requirements**
- Your **Colab notebook** should:
  - Simulate the synthetic dataset and apply Ridge and Lasso regression.
  - Plot **Bias²**, **Variance**, and **MSE** against $\lambda$ for both models.
  - Include a section answering the questions.
  - Be well-documented with comments and explanations for each step.


## **Publish on GitHub**
- Upload your Colab notebook to your **GitHub repository** for this course.
- In your repository’s **README**, include a **link** to the notebook.
- In the notebook include **“Open in Colab”** badge so the notebook can be launched directly from GitHub.


In [ ]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

coef_true=[0]*50
for i in range(50):
  if i%5!=0:
    coef_true[i]=0
  elif i==0:
    coef_true[i]=random.uniform(0,1)
  elif i%10==0:
    coef_true[i]=random.uniform(0,1)
  else:
    coef_true[i]=random.uniform(1,2)
coef_true=np.array(coef_true)



In [ ]:
print("pupa")

In [ ]:
import sklearn.linear_model
from sklearn.linear_model import LassoCV, Lasso, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def artif_data_sc():
  matr=[0]*1000
  for i in range(1000):
    matr[i]=[random.uniform(0,i+1) for j in range(50)]
    matr[i][0] = 1
  y_true = np.dot(matr, coef_true)
  df = pd.DataFrame(matr)
  df.columns = [f"x{i}" for i in range(df.shape[1])]
  df["y"] = y_true

  df_sc = StandardScaler().fit_transform(df)
  df_sc = pd.DataFrame(df_sc, columns=df.columns)

  return [df_sc.iloc[:,:-1], df_sc.iloc[:,-1]]


In [ ]:
y_true = artif_data_sc()[1]
y_true

,y
0,-1.590177
1,-1.589360
2,-1.585689
3,-1.575239
4,-1.575599
...,...
995,0.711606
996,2.586008
997,2.038809
998,2.777446


In [ ]:

alphas=np.logspace(-3, 3, 100)
biases_las=[[0 for i in range(100)] for j in range(100)]
biases_l2=[[0 for i in range(100)]  for j in range(100)]
variances_las=[[0 for i in range(100)]  for j in range(100)]
variances_l2=[[0 for i in range(100)]  for j in range(100)]

bias2_las=[0  for j in range(100)]
bias2_l2=[0   for j in range(100)]
var_las=[0   for j in range(100)]
var_l2=[0 for j in range(100)]

for j in range(len(alphas)):
  for i in range(100):
    matr  = artif_data_sc()[0]
    y_true = artif_data_sc()[1]
    X_train, X_test, y_train, y_test = train_test_split(matr, y_true, test_size=0.2, random_state=42)

    las_mod=Lasso(alpha=alphas[j]).fit(X_train, y_train)
    y_las_pred= las_mod.predict(X_train)
    l2_mod=Ridge(alpha=alphas[j]).fit(X_train, y_train)
    y_l2_pred= l2_mod.predict(X_train)

    y_las_mean_pred = np.mean(y_las_pred)
    y_l2_mean_pred = np.mean(y_l2_pred)

    biases_las[j][i]=np.mean(y_las_mean_pred-y_true)**2
    variances_las[j][i]=np.var(y_las_pred)
    biases_l2[j][i]=np.mean(y_l2_mean_pred-y_true)**2
    variances_l2[j][i]=np.var(y_l2_pred)

  bias2_las[j] = np.mean(biases_las[j])
  var_las = np.mean(variances_las[j])
  bias2_l2[j] =np.mean(biases_l2[j])
  var_l2 = (variances_l2[j])




In [ ]:
def plot_metric(metric_ridge, metric_lasso, title, ylabel):
    plt.plot(lambdas, metric_ridge, label='Ridge', marker='o')
    plt.plot(lambdas, metric_lasso, label='Lasso', marker='s')
    plt.xscale('log')
    plt.xlabel('Lambda (Regularization Strength)')
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.grid(True)

plt.subplot(1, 3, 1)
plot_metric(ridge_bias2, lasso_bias2, 'Bias² vs Lambda', 'Bias²')

plt.subplot(1, 3, 2)
plot_metric(ridge_var, lasso_var, 'Variance vs Lambda', 'Variance')

plt.subplot(1, 3, 3)
plot_metric(ridge_mse, lasso_mse, 'MSE vs Lambda', 'MSE')

plt.tight_layout()
plt.show()

Index(['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10',
       'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20',
       'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28', 'x29', 'x30',
       'x31', 'x32', 'x33', 'x34', 'x35', 'x36', 'x37', 'x38', 'x39', 'x40',
       'x41', 'x42', 'x43', 'x44', 'x45', 'x46', 'x47', 'x48', 'x49', 'y'],
      dtype='object')